In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import os

from transformers import BertTokenizer

from torch.utils.data import DataLoader

# **seed 고정**

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# **Hugging face dateset가져오기**

In [ ]:
# Hugging Face datasets 라이브러리 설치 (설치되지 않은 경우)
!pip install datasets

# GLUE 데이터셋 불러오기
from datasets import load_dataset

# MRPC (Microsoft Research Paraphrase Corpus) 태스크 로드 예시
dataset = load_dataset("glue", "mrpc")

In [ ]:
# 데이터셋 정보 출력
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


sentence1 , sentence2의 동의어 관계 판별 ,

---
label : 1 -> 동의어 관계
label : 0 -> 동의어 관계 아님

---
idx : 고유 id



In [ ]:
# 훈련, 검증 데이터 예시 출력
print(f"Train Example: {dataset['train'][0]}")
print(f"Validation Example: {dataset['validation'][0]}")

Train Example: {'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}
Validation Example: {'sentence1': "He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .", 'sentence2': '" The foodservice pie business does not fit our long-term growth strategy .', 'label': 1, 'idx': 9}


# **토큰화**

In [ ]:
# 2. BERT 토크나이저 불러오기 (사전 학습된 BERT 모델 사용)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# 3. 토큰화 함수 정의
def tokenize_function(examples):
    # sentence1과 sentence2를 함께 토큰화 (Padding, Truncation 처리)
    return tokenizer(examples["sentence1"],
                     examples["sentence2"],
                     padding="max_length",
                     truncation=True,
                     max_length=128)

In [ ]:
# 4. 데이터셋에 토큰화 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# 5. 토큰화된 데이터 예시 출력
print(tokenized_dataset["train"][0])

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0, 'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 

구조 :

---

(sentence1 원문 , sentence2 원문 , label , idx , input_ids1 & 2 , token_type_ids , attention_mask)

---
token_type_ids : 두 문장(문장1, 문장2)을 구분하기 위한 인코딩 (BERT에서 사용),

---

attention_mask : 패딩된 부분을 무시하도록 마스킹 처리 (1은 유효한 토큰, 0은 패딩)


# **배치화**

In [ ]:
# 데이터셋을 DataLoader에 넣기
train_loader = DataLoader(tokenized_dataset['train'], batch_size=8, shuffle=True)

In [ ]:
# 배치 단위로 데이터 확인
for batch in train_loader:
    print(batch)
    break  # 첫 번째 배치만 출력

{'sentence1': ['His next play , " Will Success Spoil Rock Hunter ? " a satire on Hollywood , lasted more than a year on Broadway .', "The center 's president , Joseph Torsella , was struck on the head but was able to walk to an ambulance .", 'In December 1998 , a 33-year-old woman died when she was hit by a metal cleat that came loose from the Columbia sailing ship .', 'The Production Index rose by 1.4 percentage points from 51.5 percent in May to 52.9 percent in June .', '" These are dark days for our industry , " Giovanni Bisignani , director general of the Geneva-based organization , said in a statement .', 'British-born astronaut Michael Foale is about to fly into space to take up his post as commander of the International Space Station .', 'CIA Director George Tenet said the two men were " defined by dedication and courage . "', 'He also reaffirmed his wish to resolve the North Korean nuclear crisis peacefully .'], 'sentence2': ['His next play , Will Success Spoil Rock Hunter ? , 

# **데이터 처리**

In [ ]:
# token embeddings + segment embeddings + positional embeddings

# **Encoder**

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, embedding_dim = 512 , num_heads = 8):
    super(MultiHeadAttention, self).__init__()
    self.embedding_dim = embedding_dim
    self.num_heads = num_heads
    self.head_dim = embedding_dim // num_heads

    self.q = nn.Liear(embedding_dim , embedding_dim)
    self.k = nn.Liear(embedding_dim , embedding_dim)
    self.v = nn.Liear(embedding_dim , embedding_dim)

    self.fc = nn.Linear(self.head_dim , embedding_dim)

  def forward(self , x):
    batch_size = x.size(0)

    q = self.q(x)
    k = self.k(x)
    v = self.v(x)

    q = q.view(batch_size , -1 , self.num_heads , self.head_dim).transpose(1,2) # batch , sequence_length , 8 , 512//8 = 64
    k = k.view(batch_size , -1 , self.num_heads , self.head_dim).transpose(1,2)
    v = v.view(batch_size , -1 , self.num_heads , self.head_dim).transpose(1,2)

    # Scaled Dot-Product Attention
    attention_score = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
    attention_score = torch.softmax(attention_score, dim=-1)
    attention = torch.matmul(attention_score , v)
    attention = attention.transpose(1,2).contiguous().view(batch_size , -1 , self.embedding_dim) # batch , 8 , sequence_length , 512 // 8 = 64
    output = self.fc(attention)

    return output # batch , 8 , sequence_length , 512 // 8 = 64

In [ ]:
class Encoder(nn.Module):
    def __init__(self, embed_size, num_heads, hidden_size, num_layers, vocab_size, max_length, dropout=0.1):
        super(Encoder, self).__init__()
        self.embed_size = embed_size

        # Embedding layer
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        N, seq_length = x.shape

        # Create position embeddings
        positions = torch.arange(0, seq_length).unsqueeze(0).expand(N, seq_length).to(x.device)
        out = self.word_embedding(x) + self.position_embedding(positions)
        out = self.dropout(out)
        return out